In [16]:
import xml.etree.ElementTree as ET
import pandas as pd

from google.cloud import bigquery
# from google.cloud import bigquery_storage

from time import sleep
import pandas as pd
# import table_schema as table_schema
import pytz
# import qfrm
from datetime import datetime,timedelta
# from send_email import send_error_email
# from google.api_core.exceptions import BadRequest
# from google.cloud import secretmanager
from tqdm import tqdm, tqdm_notebook
import os
tqdm_notebook().pandas()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.optimize as optimize
sns.set()
from dateutil.relativedelta import relativedelta
import math
from pandas import NaT

<ipython-input-16-ae7adc059e03>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


/Users/FICC.AI/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [64]:
os.listdir()

['ice_data_loader.ipynb',
 'sink_schedule.ipynb',
 'events.ipynb',
 'feature_engineering.ipynb',
 'amt_outstanding_change_history.ipynb',
 'yield_to_worst.ipynb',
 'call_schedule.ipynb',
 'floating_rate_history.ipynb',
 'partial_redemption_schedule.ipynb',
 '.ipynb_checkpoints',
 'default_status.ipynb',
 'organization_instrument_roles.ipynb']

In [17]:
import os

In [18]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../creds.json"

In [19]:
from lxml import etree
import pandas as pd

In [20]:
path = "../../gsm_init_muni_APFICC_GSMF10I.1.1_1.20201203T1300-05.xml"

In [21]:
from io import StringIO, BytesIO

In [22]:
from google.cloud import bigquery
import numpy as np
import xml.etree.ElementTree as ET

In [51]:
PROJECT = 'eng-reactor-287421'
dataset = 'FULL_ICE'
table = 'reference_data'

In [24]:
bqclient = bigquery.Client(project=PROJECT,)

In [25]:
def create_table_with_schema(bq,project_id,dataset,table_id,schema = []):
    PROJECT = project_id
    bq = bq
    table_id = '{}.{}.{}'.format(PROJECT,dataset,table_id)
    table = bq.create_table(table_id, exists_ok=True)
    print('{} created on {}'.format(table.table_id, table.created))
    table = bq.get_table(table_id)
    table.schema = schema
    table = bq.update_table(table, ["schema"])

def load_data(bq,data,project,dataset,table):
    bq = bq
    table_id = '{}.{}.{}'.format(project,dataset,table)
    job_config = bigquery.LoadJobConfig(schema =[])
    job = bq.load_table_from_dataframe(data, table_id,job_config=job_config)

    try:
        job.result() # Waits for the job to complete.
        return 'success'  
    except BadRequest as ex:
        print(ex) 

In [52]:
create_table_with_schema(bqclient,PROJECT,dataset,table,)
# load_data(bqclient,doc_df,PROJECT,dataset,table)

reference_data created on 2020-12-15 17:45:40.612000+00:00


In [17]:
def classification(elem,new_dict): 
    list_of_elem = elem.xpath("//instrument/global_information/organization_information/classifications")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.attrib["classification_ranking"] + "_" + element.tag:element.text})
def ratings(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/ratings")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
                tag = "_".join([element.attrib["agency"].replace("'",""),element.attrib["type"]])
                new_dict.update({tag.replace(" ","_"):element.text})
                new_dict.update({tag+"_"+"effective_date":element.attrib["effective_date"]})
    
def instrument_xref(elem,new_dict):
    for element in elem.xpath("//instrument/master_information/instrument_xref/xref"):
#         print(element.attrib)
        if element.attrib["type"] == "CUSIP":
            new_dict.update({element.attrib["type"]:element.text})
        elif element.attrib["type"] == "ISIN":
            new_dict.update({element.attrib["type"]:element.text})
        elif element.attrib["type"] == "Bloomberg Global Id":
            new_dict.update({element.attrib["type"]:element.text})
            if "id_bb_sec_num" in element.attrib:
                new_dict.update({"id_bb_sec_num":element.attrib["id_bb_sec_num"]})
            new_dict.update({"market_sector":element.attrib["market_sector"]})
            if "security_typ" in element.attrib:
                new_dict.update({"security_typ":element.attrib["security_typ"]})
            if "security_typ2" in element.attrib:
                new_dict.update({"security_typ2":element.attrib["security_typ2"]})
            if "ticker" in element.attrib:
                new_dict.update({"ticker":element.attrib["ticker"]})
            if "unique_id" in element.attrib:
                new_dict.update({"bloomberg_unique_id":element.attrib["unique_id"]})
            
def instrument_master(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/master_information/instrument_master")
#     print(len(elem.xpath("//instrument/master_information/instrument_master")[0].getchildren()))
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
        
def country_information(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/country_information/instrument_country_information")
    if len(list_of_elem)>0:
         for element in list_of_elem[0].getchildren():
                new_dict.update({element.tag:element.text})

def instrument_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/instrument_details/maturity_details")
    if len(list_of_elem)>0:
         for element in list_of_elem[0].getchildren():
                new_dict.update({element.tag:element.text})
                
def denomination_amount(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/instrument_details/denomination_amounts")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def debts_put_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/put_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def escrow_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/muni_escrow")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
def organization_master(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/master_information/organization_master")
    if len(list_of_elem)>0:
        new_dict.update({"organization_master_id":list_of_elem[0].attrib["id"]})
        for element in list_of_elem[0].getchildren():
            if element.tag != "organization_xref":
                new_dict.update({"issuer"+"_"+element.tag:element.text})
            else:
                for childs in element.getchildren():
                    new_dict.update({childs.attrib["entity_level"] +"_"+ childs.attrib["type"]:childs.text})
                    
def debts_fixed_income(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/fixed_income")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})

        
def debts_muni_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/muni_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            if element.tag != "linkage_child":
                new_dict.update({element.tag:element.text})
                
def debts_call_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/call_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            if element.tag != "call_schedule":
                new_dict.update({element.tag:element.text})
                
def debts_floating_rate_instruments(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/floating_rate_instruments")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def debts_tender_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/tender_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def debts_loc_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/loc_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})

        
def debts_sink_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/sink_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
#             print(element.text)
            new_dict.update({element.tag:element.text})
    
def market_master(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/master_information/market_master/market")
    if len(list_of_elem)>0:
        if "primary" in list_of_elem[0].attrib:
            new_dict.update({"primary_market":"true"})
        else:
            new_dict.update({"primary_market":"false"})
            
        new_dict.update({"market_id":list_of_elem[0].attrib["id"]})
        for element in list_of_elem[0].getchildren():
            if element.tag != "xref":
                new_dict.update({element.tag:element.text})
            else:
                new_dict.update({element.attrib["type"]:element.text})
                
def debts_amount_outstanding_history(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/amount_outstanding_history/change")
    if len(list_of_elem)>0:
        new_dict.update({"amount_outstanding_change_date":list_of_elem[0].attrib["date"]})
        for elements in list_of_elem[0].getchildren():
            new_dict.update({elements.tag:elements.text})


    
    
    

    
    
    
           
    
    
    
                
        
    
    

In [18]:
def create_df(new_elem,final_list):
    new_dict = {}
    
    instrument_xref(new_elem,new_dict)
    instrument_master(new_elem,new_dict)
    classification(new_elem,new_dict)
    ratings(new_elem,new_dict)
    country_information(new_elem,new_dict)
    instrument_details(new_elem,new_dict)
    denomination_amount(new_elem,new_dict)
    debts_put_details(new_elem,new_dict)
    escrow_details(new_elem,new_dict)
    organization_master(new_elem,new_dict)
    debts_fixed_income(new_elem,new_dict)
    debts_muni_details(new_elem,new_dict)
    debts_call_details(new_elem,new_dict)
    debts_floating_rate_instruments(new_elem,new_dict)
    debts_tender_details(new_elem,new_dict)
    debts_loc_details(new_elem,new_dict)
    debts_sink_details(new_elem,new_dict)
    market_master(new_elem,new_dict)
    debts_amount_outstanding_history(new_elem,new_dict)
    
    
    
    
    
    
    final_list.append(new_dict)
    

            
            
            
            
            
            
            
            
            
            
#             {'id_bb_sec_num': 'TX ANLSCD 0 08/15/2020', 'market_sector': 'Muni', 'security_typ': 'TAX CREDIT', 
#            x  'security_typ2': 'Muni', 'ticker': 'ANLSCD', 
#              'type': 'Bloomberg Global Id', 'type_id': '20', 'unique_id': 'FF000E3E987'}
            
    

In [29]:
%timeit
path2 = "../../gsm_FICC_MuniSample.792Cusips.xml"
tree = ET.parse(path2)
root = tree.getroot()[1]
time_stamp = tree.getroot()[0].find(".//timestamp").text
# doc_df = pd.DataFrame(list(iter_instrument(root,new_list_columns,count)))
doc_df = pd.DataFrame(list(iter_docs(root,time_stamp)))
final_columns = []
for columns in doc_df.columns:
    columns = columns.replace("&","")
    columns = columns.replace(" ","_")
    columns = columns.replace("-","_")
    final_columns.append(columns)


In [30]:
# "max_notification_days" in final_columns

In [31]:
# final_columns = final_columns.append("max_notification_days")

In [32]:
final_columns.append("cusip_change")
final_columns.append("hybrid_type")
final_columns.append("additional_state_code")
final_columns.append("max_notification_days")
final_columns.append("clearing_agents")
final_columns.remove("group")
len(final_columns)

212

In [33]:
# final_columns.append("additional_state_code")
len(final_columns)

212

In [79]:
final_columns

['instrument_id',
 'data_timestamp',
 'apex_asset_type',
 'instrument_type',
 'primary_name',
 'delivery_date',
 'issue_price',
 'settlement_date',
 'issue_date',
 'outstanding_indicator',
 'primary_currency_code',
 'child_issue_ind',
 'federal_tax_status',
 'sector',
 'sector_class',
 'category',
 'sub_category',
 'Moodys_Long_Rating',
 'Moodys_Long_Rating_effective_date',
 'Moodys_Credit_Watch_Long_Rating',
 'Moodys_Credit_Watch_Long_Rating_effective_date',
 'Moodys_Issue_Long_Rating',
 'Moodys_Issue_Long_Rating_effective_date',
 'country_code',
 'maturity_amount',
 'denom_increment_amount',
 'min_denom_amount',
 'organization_master_id',
 'issuerissuer',
 'issuer_incorporated_state_code',
 'issuer_primary_name',
 'issuer_primary_name_abbreviated',
 'issuer_organization_status',
 'issuer_organization_type',
 'issuer_org_country_code',
 'id_bb_sec_num',
 'market_sector',
 'security_typ',
 'ticker',
 'unique_id',
 'accrual_date',
 'bond_form',
 'bond_insurance',
 'coupon_type',
 'curre

In [37]:
"clearing_agents" in final_columns

True

In [ ]:
# final_columns

In [19]:
def threaded_stuff(final_list):
    doc_df = pd.DataFrame(final_list)
    new_columns = []
    for columns in doc_df.columns:
        columns = columns.replace("&","")
        columns = columns.replace(" ","_")
        columns = columns.replace("-","_")
        new_columns.append(columns)
    doc_df.columns = new_columns
    for columns in list(set(final_columns) - set(new_columns)):
        doc_df[columns] = np.nan
    for columns in doc_df.columns:
        doc_df[columns] = doc_df[columns].astype('str')
    if "Ticker" in doc_df.columns:
        del doc_df["Ticker"]
#             for cols in doc_df.columns:
#                 print(cols,end = " ")
#             print("\n")


#             print(doc_df.tail())
    print(len(doc_df))
    
    try:
        test = load_data(bqclient,doc_df,PROJECT,dataset,table)
        print(test)
    except Exception as e:
        print(e)
        for columns in doc_df.columns:
            print(columns)
        print(doc_df.head)
        print(count)
   

In [21]:
import multiprocessing

In [54]:
def load_dataframe(bqclient,doc_df,PROJECT,dataset,table,count):
    try:
        test = load_data(bqclient,doc_df,PROJECT,dataset,table)
        print(test)
    except Exception as e:
        print(e)
        for columns in doc_df.columns:
            print(columns)
        print(doc_df.head)
        print(count)
    

In [26]:
storage_path = "../../data_split/"

In [24]:
def call_schedule(new_elem,final_list):
    new_dict = {}
    list_of_elem = new_elem.xpath("//instrument/debt/call_details")
    instrument_id = new_elem.xpath("//instrument")[0].attrib["id"]
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            if element.tag == "call_schedule":
                print("inside call schedule")
#                 print(len(element.getchildren()))
#                 new_dict.update({element.tag:element.text})
                for childs in element.getchildren():
#                     print("inside childs")
#                     print(childs.text)
#                     print(childs.text)
                    new_dict.update({"instrument_id":instrument_id})
                    new_dict.update({childs.tag:childs.text})
                    
                    final_list.append(new_dict)
                    
       
    
#                     new_dict.update({element.find(".//call_date").tag:element.find(".//call_date").text})
#                     if element.find(".//cav_call_price") is not None:
#                         new_dict.update({element.find(".//cav_call_price").tag:element.find(".//cav_call_price").text})
                    
    

In [25]:
def create_df_call_schedule(new_elem,final_list):
#     new_dict = {}
    
    call_schedule(new_elem,final_list)
#     print(new_dict)
    

In [60]:
# def get_sink_schedule(author):
#     for doc in author.iter('instrument'):
#         sink_schedule = doc.iterfind(".//debt/sink_schedule")
#         if sink_schedule is not None:
#             for element in sink_schedule:
#                 new_dict = {}
#                 new_dict.update({"instrument_id":doc.attrib["id"]})
#                 if element.find(".//mandatory_sink_amount") is not None:
#                     new_dict.update({element.find(".//mandatory_sink_amount").tag:element.find(".//mandatory_sink_amount").text})
#                 if element.find(".//mandatory_sink_date") is not None:
#                     new_dict.update({element.find(".//mandatory_sink_date").tag:element.find(".//mandatory_sink_date").text})
#                 if element.find(".//mandatory_sink_price") is not None:
#                     new_dict.update({element.find(".//mandatory_sink_price").tag:element.find(".//mandatory_sink_price").text})


In [ ]:
def sink_schedule(new_elem):
    new_dict = {}
    list_of_elem = new_elem.xpath("//instrument/debt/sink_schedule")
    instrument_id = new_elem.xpath("//instrument")[0].attrib["id"]
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
#             if element.tag == "call_schedule":
#                 print("inside call schedule")
#                 print(len(element.getchildren()))
#                 new_dict.update({element.tag:element.text})
#                 for childs in element.getchildren():
#                     print("inside childs")
#                     print(childs.text)
#                     print(childs.text)
            new_dict.update({"instrument_id":instrument_id})
            new_dict.update({childs.tag:childs.text})
#         print(new_dict)
    return new_dict
    

In [ ]:
def create_df_sink_schedule(new_elem,final_list):
#     new_dict = {}
    
    new_dict = sink_schedule(new_elem,)
#     print(new_dict)
    
    
    if len(new_dict)>0:
        final_list.append(new_dict)

In [26]:
# count = 0
total = 1086089


def new_fast_iter(context, *args, **kwargs):
  
    processes = []
    final_list = []
    count = 0
    jobs = []
    for event,elem in context:
        process_element(elem,final_list)
        count += 1
    #         elem.clear()
    #         elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
#         if count == 10:
#             break
        print("******")
        if count == 10:
            break

        if count%10000 == 0:
    #             print(elem.attrib)
            print(count)
        if count %30000== 0:
#             thread = threading.Thread(target = threaded_stuff(final_list))
            doc_df = pd.DataFrame(final_list)
#             print(doc_df)
            new_columns = []
            for columns in doc_df.columns:
                columns = columns.replace("&","")
                columns = columns.replace(" ","_")
                columns = columns.replace("-","_")
                new_columns.append(columns)
            doc_df.columns = new_columns
            doc_df.replace(np.nan,None,inplace = True)
            doc_df.replace([np.nan],[None],inplace = True)
            doc_df.replace("nan",None,inplace = True)
            print(len(doc_df))
            
#             doc_df.to_pickle(storage_path+"_"+str(count)+".pkl")
            load_dataframe(bqclient,doc_df,PROJECT,dataset,table,count)
            final_list = []


        if count == total:
            doc_df = pd.DataFrame(final_list)
            new_columns = []
            for columns in doc_df.columns:
                columns = columns.replace("&","")
                columns = columns.replace(" ","_")
                columns = columns.replace("-","_")
                new_columns.append(columns)
            doc_df.columns = new_columns
            doc_df.replace(np.nan,None,inplace = True)
            doc_df.replace([np.nan],[None],inplace = True)
            doc_df.replace("nan",None,inplace = True)
            print(len(doc_df))

            load_dataframe(bqclient,doc_df,PROJECT,dataset,table,count)
    
            final_list = []
            

def process_element(elem,final_list):
#     new_elem = elem
    
    new_elem = etree.parse(BytesIO(etree.tostring(elem)))
#     print(etree.tostring(elem, pretty_print=True))
#     print(etree.dump(new_elem))
    create_df_call_schedule(new_elem,final_list)



context = etree.iterparse(path,events=("start",),tag = "instrument")
new_fast_iter(context)

******
******
{}
******
******
******
******
******
******
******
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
inside call schedule
{'instrument_id': '279575419', 'call_date': '2036-01-15', 'call_price': '100.0000000'}
******


In [35]:
def classification(elem,new_dict): 
    list_of_elem = elem.xpath("//instrument/global_information/organization_information/classifications")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.attrib["classification_ranking"] + "_" + element.tag:element.text})
def ratings(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/ratings")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
                tag = "_".join([element.attrib["agency"].replace("'",""),element.attrib["type"]])
                new_dict.update({tag.replace(" ","_"):element.text})
                new_dict.update({tag+"_"+"effective_date":element.attrib["effective_date"]})
    
def instrument_xref(elem,new_dict):
    for element in elem.xpath("//instrument/master_information/instrument_xref/xref"):
#         print(element.attrib)
        if element.attrib["type"] == "CUSIP":
            new_dict.update({element.attrib["type"]:element.text})
        elif element.attrib["type"] == "ISIN":
            new_dict.update({element.attrib["type"]:element.text})
        elif element.attrib["type"] == "Bloomberg Global Id":
            new_dict.update({element.attrib["type"]:element.text})
            if "id_bb_sec_num" in element.attrib:
                new_dict.update({"id_bb_sec_num":element.attrib["id_bb_sec_num"]})
            new_dict.update({"market_sector":element.attrib["market_sector"]})
            if "security_typ" in element.attrib:
                new_dict.update({"security_typ":element.attrib["security_typ"]})
            if "security_typ2" in element.attrib:
                new_dict.update({"security_typ2":element.attrib["security_typ2"]})
            if "ticker" in element.attrib:
                new_dict.update({"ticker":element.attrib["ticker"]})
            if "unique_id" in element.attrib:
                new_dict.update({"bloomberg_unique_id":element.attrib["unique_id"]})
            
def instrument_master(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/master_information/instrument_master")
#     print(len(elem.xpath("//instrument/master_information/instrument_master")[0].getchildren()))
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
        
def country_information(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/country_information/instrument_country_information")
    if len(list_of_elem)>0:
         for element in list_of_elem[0].getchildren():
                new_dict.update({element.tag:element.text})

def instrument_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/instrument_details/maturity_details")
    if len(list_of_elem)>0:
         for element in list_of_elem[0].getchildren():
                new_dict.update({element.tag:element.text})
                
def denomination_amount(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/global_information/instrument_details/denomination_amounts")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def debts_put_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/put_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def escrow_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/muni_escrow")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
def organization_master(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/master_information/organization_master")
    if len(list_of_elem)>0:
        new_dict.update({"organization_master_id":list_of_elem[0].attrib["id"]})
        for element in list_of_elem[0].getchildren():
            if element.tag != "organization_xref":
                new_dict.update({"issuer"+"_"+element.tag:element.text})
            else:
                for childs in element.getchildren():
                    new_dict.update({childs.attrib["entity_level"] +"_"+ childs.attrib["type"]:childs.text})
                    
def debts_fixed_income(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/fixed_income")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})

        
def debts_muni_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/muni_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            if element.tag != "linkage_child":
                new_dict.update({element.tag:element.text})
                
def debts_call_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/call_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            if element.tag != "call_schedule":
                new_dict.update({element.tag:element.text})
                
def debts_floating_rate_instruments(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/floating_rate_instruments")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def debts_tender_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/tender_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})
            
def debts_loc_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/loc_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            new_dict.update({element.tag:element.text})

        
def debts_sink_details(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/sink_details")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
#             print(element.text)
            new_dict.update({element.tag:element.text})
    
def market_master(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/master_information/market_master/market")
    if len(list_of_elem)>0:
        if "primary" in list_of_elem[0].attrib:
            new_dict.update({"primary_market":"true"})
        else:
            new_dict.update({"primary_market":"false"})
            
        new_dict.update({"market_id":list_of_elem[0].attrib["id"]})
        for element in list_of_elem[0].getchildren():
            if element.tag != "xref":
                new_dict.update({element.tag:element.text})
            else:
                new_dict.update({element.attrib["type"]:element.text})
                
def debts_amount_outstanding_history(elem,new_dict):
    list_of_elem = elem.xpath("//instrument/debt/amount_outstanding_history/change")
    if len(list_of_elem)>0:
        new_dict.update({"amount_outstanding_change_date":list_of_elem[0].attrib["date"]})
        for elements in list_of_elem[0].getchildren():
            new_dict.update({elements.tag:elements.text})


    
    
    

    
    
    
           
    
    
    
                
        
    
    

In [36]:
str_column = ["additional_state_code","instrument_id","unique_id",]
del_column = ['Moodys_Enhanced_Short_Rating_effective_date','data_timestamp',
 'Moodys_Credit_Watch_Short_Rating',
 'Standard__Poors_School_ICR_Long_Rating',
 'Moodys_Enhanced_Long_Rating_effective_date',
 'Standard__Poors_Short_Rating',
 'Fitch_Ratings_Credit_Watch_Long_Rating',
 'Fitch_Ratings_Issue_Long_Rating_effective_date',
 'Moodys_Long_Rating',
 'Moodys_Enhanced_Long_Rating',
 'Moodys_Issue_Long_Rating_effective_date',
 'Standard__Poors_Short_Rating_effective_date',
 'Standard__Poors_Credit_Watch_Long_Rating_effective_date',
 'Moodys_Insured_Long_Rating',
 'Moodys_Issue_Short_Rating',
 'Standard__Poors_Issue_Long_Rating',
 'Fitch_Ratings_Issue_Short_Rating_effective_date',
 'Moodys_Credit_Watch_Long_Rating_effective_date',
 'Moodys_Enhanced_Short_Rating',
 'Fitch_Ratings_Short_Rating',
 'Standard__Poors_Credit_Watch_Long_Outlook_Rating_effective_date',
 'Moodys_Credit_Watch_Short_Rating_effective_date',
 'Fitch_Ratings_Short_Rating_effective_date',
 'Fitch_Ratings_Long_Rating_effective_date',
 'Moodys_Short_Rating',
 'Standard__Poors_Long_Rating',
 'Fitch_Ratings_Long_Rating',
 'Fitch_Ratings_Issue_Short_Rating',
 'Moodys_Credit_Watch_Long_Rating',
 'Standard__Poors_Credit_Watch_Long_Rating',
 'Fitch_Ratings_Issue_Long_Rating',
 'Moodys_Long_Rating_effective_date',
 'Standard__Poors_School_ICR_Long_Rating_effective_date',
 'Moodys_Short_Rating_effective_date',
 'Moodys_Issue_Long_Rating',
 'Moodys_Insured_Long_Rating_effective_date',
 'Standard__Poors_Long_Rating_effective_date',
 'Fitch_Ratings_Credit_Watch_Long_Rating_effective_date',
 'Standard__Poors_Issue_Long_Rating_effective_date',
 'Standard__Poors_Credit_Watch_Long_Outlook_Rating',
 'Moodys_Issue_Short_Rating_effective_date',]
question_columns = ["SEDOL","linked_markets",
    "clearing_agents",
    "sector_class",
    "category",
    "max_notification_days",
    "NAICS_primary",
    "cusip_change",
    "SIC_primary",
    "issuerissuer",
    "SEDOL_status",
    "sub_category",
    "hybrid_type",
    "IDII_status","sector",]
# integer_columns = []
float_columns = ["maturity_amount","denom_increment_amount","min_denom_amount","issue_price","orig_principal_amount",
                "amount_outstanding", "amount_outstanding_decimal",]
date_columns = ["next_coupon_payment_date","delivery_date","settlement_date","issue_date","accrual_date","first_coupon_date", "previous_coupon_payment_date", "first_execution_date", 
                "last_period_accrues_from_date","maturity_date","formal_award_date", "sale_date", "next_call_date", "par_call_date", "next_sink_date",
                "amount_outstanding_change_date", "other_accrual_date", "refund_date","refunding_dated_date",
                "orig_avg_life_date","current_coupon_reset_date","previous_coupon_effective_date","next_put_date",
                "put_end_date","put_start_date","first_variable_reset_date","make_whole_call_end_date", 
                "next_reset_date","loc_expiration_date","odd_first_coupon_date","next_tender_date", 
                "tender_start_date","designated_termination_date","conditional_call_date","next_auction_reset_date",]

In [50]:
def create_df(new_elem,final_list):
    new_dict = {}
    list_of_elem = new_elem.xpath("//instrument")
    new_dict.update({"instrument_id":list_of_elem[0].attrib["id"]})
    
    instrument_xref(new_elem,new_dict)
    instrument_master(new_elem,new_dict)
    classification(new_elem,new_dict)
    ratings(new_elem,new_dict)
    country_information(new_elem,new_dict)
    instrument_details(new_elem,new_dict)
    denomination_amount(new_elem,new_dict)
    debts_put_details(new_elem,new_dict)
    escrow_details(new_elem,new_dict)
    organization_master(new_elem,new_dict)
    debts_fixed_income(new_elem,new_dict)
    debts_muni_details(new_elem,new_dict)
    debts_call_details(new_elem,new_dict)
    debts_floating_rate_instruments(new_elem,new_dict)
    debts_tender_details(new_elem,new_dict)
    debts_loc_details(new_elem,new_dict)
    debts_sink_details(new_elem,new_dict)
    market_master(new_elem,new_dict)
    debts_amount_outstanding_history(new_elem,new_dict)
#     print("next_call_date" in new_dict)
    
    
    
    
    
    
    final_list.append(new_dict)

In [23]:
create_table_with_schema(bqclient,PROJECT,dataset,table,)

NameError: name 'create_table_with_schema' is not defined

In [38]:
import pytz
from datetime import datetime
my_timezone = pytz.timezone('America/New_York')


In [175]:
#  in final_columns

In [55]:
def upload_dataframe(final_list,count,data_timestamp):
    doc_df = pd.DataFrame(final_list)
    new_columns = []
    for columns in doc_df.columns:
        columns_list.append(columns)
        columns = columns.replace("&","")
        columns = columns.replace(" ","_")
        columns = columns.replace("-","_")
        new_columns.append(columns)
    doc_df.columns = new_columns
    for columns in list(set(final_columns) - set(new_columns)):
#                 columns_list.append(columns)
        doc_df[columns] = None
#     
    doc_df["upload_date"] = my_timezone.localize(datetime.now()).date() 
    
#     for columns in del_column:
#         del doc_df[columns]
    doc_df.replace(np.nan,None,inplace = True)
    for columns in float_columns:
        doc_df[columns] = doc_df[float_columns].astype("float")
        doc_df[columns] = pd.Series(doc_df[columns]).round(4)
    for columns in question_columns:
        doc_df[columns] = doc_df[columns].astype('float')
    for columns in str_column:
        doc_df[columns] = doc_df[columns].astype('str')
        


    for columns in doc_df.columns:
        if columns in date_columns:
            doc_df[columns] = pd.to_datetime(doc_df[columns],format = "%Y-%m-%d",exact = True,errors = 'coerce').dt.date

    doc_df["data_timestamp"] = pd.to_datetime(data_timestamp[0])
    doc_df["data_timestamp"] = doc_df["data_timestamp"].dt.date
#             for columns in doc_df.columns:
#                 doc_df[columns] = doc_df[columns].astype('str')
    if "Ticker" in doc_df.columns:
        del doc_df["Ticker"]
    


#             print(len(doc_df))

#             print(doc_df.head)
#             break
#             doc_df.to_pickle(storage_path+"_"+str(count)+".pkl")
    load_dataframe(bqclient,doc_df,PROJECT,dataset,table,count)

In [40]:
final_columns = list(set(final_columns) - set(del_column))

In [41]:
len(final_columns)

171

In [41]:
len(final_columns)

171

hello


In [44]:
# count = 0
# total = 0

# for j,k in context:
#     count += 1
#     if count % 30000 == 0:
#         print(count)
#     while k.getprevious() is not None:    
#         del k.getparent()[0]
# else:
#     print("done")
#     total = count

# print(total)
        

In [14]:
context = etree.iterparse(path,events=("start",),tag = "instrument")
# count = 0
# for event,elem in context:
#     if count%20000 == 0:
#         print(count)
#     count+=1

In [ ]:
# count = 0
total = 1086089
columns_list = []
data_timestamp = []
head_context = etree.iterparse(path,events=("start",),tag = "header")
for event,elem in head_context:
    new_elem = etree.parse(BytesIO(etree.tostring(elem)))
    list_of_elem = elem.xpath("//header")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
#             print(element.tag)
            if element.tag == "timestamp":
               
                data_timestamp.append(element.text)
    break



def fast_iter(context,data_timestamp, *args, **kwargs):
  
    processes = []
    final_list = []
    count = 0
    jobs = []
    for event,elem in context:
        process_element(elem,final_list)
        count += 1
    #         elem.clear()
    #         elem.clear()
        while elem.getprevious() is not None:
            
            del elem.getparent()[0]


        if count%10000 == 0:
    #             print(elem.attrib)
            print(count)
        if count%30000 == 0:
            upload_dataframe(final_list,count,data_timestamp)
#             thread = threading.Thread(target = threaded_stuff(final_list))
            
            final_list = []

        if count == total:
            upload_dataframe(final_list,count,data_timestamp)
            final_list = []
            
        


def process_element(elem,final_list):
#     new_elem = elem
    
    new_elem = etree.parse(BytesIO(etree.tostring(elem)))
#     print(etree.tostring(new_elem, pretty_print=True))
#     print("*************************************")
#     print(etree.dump(new_elem))
    create_df(new_elem,final_list)



context = etree.iterparse(path,events=("start",),tag = "instrument")
fast_iter(context,data_timestamp)

10000
20000
30000
success
40000
50000
60000
success
70000
80000
90000
success
100000
110000
120000
success
130000
140000
150000
success
160000
170000
180000
success
190000
200000
210000
success
220000
230000
240000
success
250000
260000
270000
success
280000
290000
300000
success
310000
320000
330000
success
340000
350000
360000
success
370000
380000
390000
success
400000
410000
420000
success
430000
440000
450000
success
460000
470000
480000
success
490000
500000
510000
success
520000
530000
540000
success
550000
560000
570000
success
580000
590000
600000
success
610000
620000
630000
success
640000
650000
660000
success
670000
680000
690000
success
700000
710000
720000
success
730000
740000
750000
success
760000
770000
780000
success
790000
800000
810000
success
820000
830000
840000
success
850000
860000
870000
success
880000
890000
900000
success
910000
920000
930000
success
940000
950000
960000
success
970000
980000
990000
success
1000000
1010000
1020000
success
1030000
1040000


type
timestamp
<built-in method date of Timestamp object at 0x7fb2781b6e40>


In [ ]:
# # count = 0

# def fast_iter(context, *args, **kwargs):
#     workers = 10
#     final_list = []
#     count = 0
#     for event,elem in context:
#         process_element(elem,final_list)
#         count += 1
#     #         elem.clear()
#     #         elem.clear()
#         while elem.getprevious() is not None:
#             del elem.getparent()[0]

#         if count%10000 == 0:
#     #             print(elem.attrib)
#             print(count)
#         if count%30000 == 0:
#             doc_df = pd.DataFrame(final_list)
#             new_columns = []
#             for columns in doc_df.columns:
#                 columns = columns.replace("&","")
#                 columns = columns.replace(" ","_")
#                 columns = columns.replace("-","_")
#                 new_columns.append(columns)
#             doc_df.columns = new_columns
#             for columns in list(set(final_columns) - set(new_columns)):
#                 doc_df[columns] = np.nan
#             for columns in doc_df.columns:
#                 doc_df[columns] = doc_df[columns].astype('str')
#             if "Ticker" in doc_df.columns:
#                 del doc_df["Ticker"]
#     #             for cols in doc_df.columns:
#     #                 print(cols,end = " ")
#     #             print("\n")


#     #             print(doc_df.tail())
#             print(len(doc_df))
#             try:
#                 test = load_data(bqclient,doc_df,PROJECT,dataset,table)
#                 print(test)
#             except Exception as e:
#                 print(e)
#                 for columns in doc_df.columns:
#                     print(columns)
#                 print(doc_df.head)
#                 print(count)
#             final_list = []

# #         print(event)
        
            
            
        


# def process_element(elem,final_list):
# #     new_elem = elem
    
#     new_elem = etree.parse(BytesIO(etree.tostring(elem)))
# #     print(etree.tostring(elem, pretty_print=True))
# #     print(etree.dump(new_elem))
#     create_df(new_elem,final_list)

# #     for element in new_elem.xpath("//instrument/master_information/instrument_xref/xref"):
# #         pass
# #         print(element.attrib)
# #         print(element.text)
# #         new_dict.update({element.attrib.type:element.text})


# context = etree.iterparse(path,events=("start",),tag = "instrument")
# fast_iter(context)

10000
20000
30000
30000
success
40000
50000
60000
30000
success
70000
80000
90000
30000
success
100000
110000
120000
30000
success
130000
140000
150000
30000
success
160000
170000
180000
30000
success
190000
200000
210000
30000
success
220000
230000
240000
30000
name 'BadRequest' is not defined
CUSIP
ISIN
apex_asset_type
instrument_type
primary_name
delivery_date
settlement_date
issue_date
outstanding_indicator
primary_currency_code
child_issue_ind
federal_tax_status
country_code
maturity_amount
organization_master_id
issuer_CUSIP_6
issuer_incorporated_state_code
issuer_primary_name
issuer_primary_name_abbreviated
issuer_organization_status
issuer_organization_type
issuer_org_country_code
accrual_date
bond_form
bond_insurance
coupon_type
daycount_basis_type
debt_type
default_indicator
depository_type
first_coupon_date
interest_payment_frequency
issue_amount
last_period_accrues_from_date
maturity_date
next_coupon_payment_date
orig_principal_amount
outstanding_amount
sale_type
settlement

250000
260000
270000
30000
success
280000
290000
300000
30000
success
310000
320000
330000
30000
success
340000
350000
360000
30000
success
370000
380000
390000
30000
success
400000
410000
420000
30000
success
430000
440000
450000
30000
success
460000
470000
480000
30000
success
490000
500000
510000
30000
success
520000
530000
540000
30000
success
550000
560000
570000
30000
name 'BadRequest' is not defined
CUSIP
ISIN
Bloomberg_Global_Id
id_bb_sec_num
market_sector
security_typ
security_typ2
ticker
bloomberg_unique_id
apex_asset_type
instrument_type
primary_name
delivery_date
issue_price
settlement_date
issue_date
outstanding_indicator
primary_currency_code
primary_exchange
child_issue_ind
eval_quotation_basis
federal_tax_status
country_code
maturity_amount
denom_increment_amount
min_denom_amount
organization_master_id
issuer_Issuer_Code
issuer_CUSIP_6
issuer_incorporated_state_code
issuer_primary_name
issuer_primary_name_abbreviated
issuer_organization_status
issuer_organization_type
i

580000
590000
600000
30000
name 'BadRequest' is not defined
CUSIP
ISIN
Bloomberg_Global_Id
id_bb_sec_num
market_sector
security_typ
security_typ2
ticker
bloomberg_unique_id
apex_asset_type
instrument_type
primary_name
delivery_date
issue_price
settlement_date
issue_date
outstanding_indicator
primary_currency_code
primary_exchange
child_issue_ind
eval_quotation_basis
federal_tax_status
country_code
maturity_amount
denom_increment_amount
min_denom_amount
organization_master_id
issuer_Issuer_Code
issuer_CUSIP_6
issuer_incorporated_state_code
issuer_primary_name
issuer_primary_name_abbreviated
issuer_organization_status
issuer_organization_type
issuer_org_country_code
accrual_date
bond_form
bond_insurance
coupon_type
current_coupon_rate
daycount_basis_type
debt_type
default_indicator
depository_type
first_coupon_date
interest_payment_frequency
issue_amount
last_period_accrues_from_date
maturity_date
next_coupon_payment_date
orig_principal_amount
original_yield
outstanding_amount
sale_type


In [28]:
#4:05
def iter_docs(author,time_stamp):
#     author_attr = author.attrib
    for doc in author.iter('instrument'):
        # print(doc.attrib["id"])
        
        new_dict = {}
        new_dict.update({"instrument_id":doc.attrib["id"]})
        new_dict.update({"data_timestamp":time_stamp})
        
        
#         doc_dict = author_attr.copy()
        # xref = doc.find("master_information").find("instrument_xref")
        # print(xref.tag)

        cusip = doc.find('.//instrument_xref/xref[@type="CUSIP"]')
        isin = doc.find('.//instrument_xref/xref[@type="ISIN"]')
        bbid = doc.find('.//instrument_xref/xref[@type="Bloomberg Global Id"]')
        instrument_master = doc.find(".//instrument_master")
        if instrument_master is not None:    
            for elements in instrument_master:
                # print(elements.tag)
                new_dict.update({elements.tag:elements.text})
        ranking_details = doc.find(".//global_information/organization_information/classifications")
        if ranking_details is not None:
            for element in ranking_details:
                new_dict.update({element.tag + "_" + element.attrib["classification_ranking"]:element.text})
                
                
                
        
        
        global_information_rating =  doc.find(".//global_information/ratings")
        if global_information_rating is not None:
            for element in global_information_rating:
                tag = "_".join([element.attrib["agency"].replace("'",""),element.attrib["type"]])
                new_dict.update({tag.replace(" ","_"):element.text})
                new_dict.update({tag+"_"+"effective_date":element.attrib["effective_date"]})

        country_information = doc.find(".//country_information/instrument_country_information")
        if country_information is not None:
            for element in country_information:
                new_dict.update({element.tag:element.text})
        # print(bbid.attrib)instrument_details
        maturity_details = doc.find(".//instrument_details/maturity_details")

        if maturity_details is not  None:
            for element in maturity_details:
                new_dict.update({element.tag:element.text})
            
        increment_details = doc.find(".//instrument_details/denomination_amounts")
        if increment_details is not None:
            for element in increment_details:
                new_dict.update({element.tag:element.text})
         #muni_escrow       
        
        put_details = doc.find(".//debt/put_details")
        if put_details is not None:
            for element in put_details:
                new_dict.update({element.tag:element.text})
                
        
        
        escrow_details = doc.find(".//instrument_details/muni_escrow")
        if escrow_details is not None:
            for element in escrow_details:
                new_dict.update({element.tag:element.text})
        
        organization_master = doc.find(".//organization_master")
        if organization_master is not None:
            new_dict.update({"organization_master_id":organization_master.attrib["id"]})
            for element in organization_master:
#                 print(element.tag)
                if element.tag != "organization_xref":
                    new_dict.update({"issuer"+"_"+element.tag:element.text})
                else:
                    for childs in element:
                        new_dict.update({childs.attrib["entity_level"] + childs.attrib["entity_level"].replace(" ","_"):childs.text})
                        
            

        # for element in maturity_details:

        for elements in ["id_bb_sec_num","market_sector","security_typ","ticker","unique_id"]:
            new_dict.update({elements:bbid.attrib[elements]})
        debt_info = doc.find(".//debt/fixed_income")
        if debt_info is not None:
            for element in debt_info:
                new_dict.update({element.tag:element.text})
           
        muni_info = doc.find(".//debt/muni_details")
        
        if muni_info is not None:
            for element in muni_info:
                if element.tag != "linkage_child":
                    new_dict.update({element.tag:element.text})
        
        call_details_info = doc.find(".//debt/call_details")
        if call_details_info is not None:
            for element in call_details_info:
                if element.tag != "call_schedule":
                    new_dict.update({element.tag:element.text})
        floating_rate_details = doc.find(".//debt/floating_rate_instruments")
        if floating_rate_details is not None:
            for element in floating_rate_details:
                new_dict.update({element.tag:element.text})
                    
        tender_details = doc.find(".//debt/tender_details")
        if tender_details is not None:
            for element in tender_details:
                new_dict.update({element.tag:element.text})
            
        loc_details = doc.find(".//debt/loc_details")
        if loc_details is not None:
            for element in loc_details:
                new_dict.update({element.tag:element.text})
                
        sink_details = doc.find(".//debt/sink_details")
        if sink_details is not None:
            for element in sink_details:
                if element == "mandatory_sink_amount_type":
                    new_dict.update({element.tag:element.attrib["type"]})
                else:
                    new_dict.update({element.tag:element.text})

                # print(element.tag)
                # new_dict.update({element.tag:element.attrib["type"]})
#         <SIC classification_ranking="primary" classification_ranking_id="1" type="Standard Industrial Classification" type_id="1">6552</SIC>

    
        market_master = doc.find(".//market_master/market")
        if market_master is not None:
            new_dict.update({"primary_market":market_master.attrib["primary"]})
        
            for element in market_master:
                if element.tag == "xref":
                    new_dict.update({element.attrib["type"]:element.text})
                    new_dict.update({element.attrib["type"]+"_"+"status":element.attrib["status"]})
                else:
                    new_dict.update({element.tag:element.text})
                    

        amt_outstanding = doc.find(".//debt/amount_outstanding_history/change")
        # print(amt_outstanding.attrib)
        if amt_outstanding is not None:
            new_dict.update({"amount_outstanding_change_date":amt_outstanding.attrib["date"]})

            for elements in amt_outstanding:
                new_dict.update({elements.tag:elements.text})

        new_dict.update({cusip.attrib["type"]:cusip.text})
        new_dict.update({isin.attrib["type"]:isin.text})
        new_dict.update({bbid.attrib["type"]:bbid.text})


#         doc_dict.update(new_dict)
        
   
        yield new_dict

In [7]:
context = etree.iterparse(path, tag='instrument' )

In [ ]:
import xml.etree.ElementTree as ET

In [200]:
import xml.etree.cElementTree as ET

In [ ]:
%timeit
tree = ET.parse(xml_data_string)
root = tree.getroot()[1]
time_stamp = tree.getroot()[0].find(".//timestamp").text
# doc_df = pd.DataFrame(list(iter_instrument(root,new_list_columns,count)))
doc_df = pd.DataFrame(list(iter_docs(root,time_stamp)))

In [1]:
from lxml import etree

import sys

In [214]:
path2 = "../../gsm_FICC_MuniSample.792Cusips.xml"

In [4]:
path = "../../gsm_init_muni_APFICC_GSMF10I.1.1_1.20201203T1300-05.xml"

In [215]:
%timeit
path2 = "../../gsm_FICC_MuniSample.792Cusips.xml"
tree = ET.parse(path2)
root = tree.getroot()[1]
time_stamp = tree.getroot()[0].find(".//timestamp").text
# doc_df = pd.DataFrame(list(iter_instrument(root,new_list_columns,count)))
doc_df = pd.DataFrame(list(iter_docs(root,time_stamp)))
final_columns = []
for columns in doc_df.columns:
    columns = columns.replace("&","")
    columns = columns.replace(" ","_")
    columns = columns.replace("-","_")
    final_columns.append(columns)

In [217]:
final_columns = []
for columns in doc_df.columns:
    columns = columns.replace("&","")
    columns = columns.replace(" ","_")
    columns = columns.replace("-","_")
    final_columns.append(columns)

In [218]:
len(final_columns)

208

In [ ]:
#1086089
sum(1 for _ in context)

In [ ]:
len(new_list)/2

In [4]:
# list_of_columns = [lambda x:xpath_modify(x) for x in list_of_columns ]list_of_columns)
# list_of_columns


In [8]:
attribute_dict = {}
def xpath_modify(url):
    if "@" in url:
        url = url.split("/")
        dict_key = url[len(url)-1].replace("@","")
        final_dict_key =  url[len(url)-2] + "_" + dict_key
        url = url[1:len(url)-1]
        url.insert(0,"./",)
        url = "/".join(url)
        attribute_dict.update({url:[dict_key,final_dict_key]})
    else:
        url = "./"+url
    return url

In [9]:
# list_of_columns = [lambda x:xpath_modify(x) for x in list_of_columns ]
new_list_columns = []
for element in list_of_columns:
    new_element = xpath_modify(element)
    new_list_columns.append(new_element)

In [120]:
# list_of_columns
attribute_dict

In [122]:
# def url_function(new_dict,url):
    # print(url)
    # if "@" in url:
    #     url = url.split("/")
    #     dict_key = url[len(url)-1].replace("@","")
    #     final_dict_key =  url[len(url)-2] + "_" + dict_key
    #     url = url[1:len(url)-1]
    #     url.insert(0,"./",)
    #     url = "/".join(url)
    #     node = root.find(url)
    #     if node is not None:
    #         if len(node) !=0:
    #             new_dict.update({final_dict_key:node.attrib[dict_key]})
    #         else:
    #             new_dict.update({node.tag:node.text})
    # else:
    #     url = "./"+url
    #     node = root.find(url)
    #     if node is not None:
    #         new_dict.update({node.tag:node.text})
    # node = root.find(url)
    # if node is not None and len(node.attrib)!=0:
    #     new_dict.update({attribute_dict[url]:node.attrib[dict_key]})
    # else:
    #     new_dict.update({node.tag:node.text})




In [10]:
def get_column_value_dictionary(instrument,new_list_columns):
     new_dict = {}
     
     for element in new_list_columns:
#         print(element)

        node = root.find(element)
        if node is not None :
            if len(node.attrib)!=0 and element in attribute_dict:
                new_dict.update({attribute_dict[element][0]:node.attrib[attribute_dict[element][0]]})
            else:
                new_dict.update({node.tag:node.text})
    
     return new_dict
    



In [11]:
def iter_instrument(instrument,new_list_columns,count):
    for node in instrument.iter('instrument'):
        count += 1
        print(count)
        starttime= datetime.now()
        # print("hello start")
        final_dict = get_column_value_dictionary(node,new_list_columns)
        endtime = datetime.now()
        print("time_taken for get get_column_value_dictionary: {i}".format(i = endtime - starttime))
        # print("hell done")
        print(len(final_dict))

        yield final_dict



In [241]:
def iter_docs(author,time_stamp):
#     author_attr = author.attrib
    for doc in author.iter('instrument'):
        # print(doc.attrib["id"])
        
        new_dict = {}
        new_dict.update({"instrument_id":doc.attrib["id"]})
        new_dict.update({"data_timestamp":time_stamp})
        
        
#         doc_dict = author_attr.copy()
        # xref = doc.find("master_information").find("instrument_xref")
        # print(xref.tag)

        cusip = doc.find('.//instrument_xref/xref[@type="CUSIP"]')
        isin = doc.find('.//instrument_xref/xref[@type="ISIN"]')
        bbid = doc.find('.//instrument_xref/xref[@type="Bloomberg Global Id"]')
        instrument_master = doc.find(".//instrument_master")
        if instrument_master is not None:    
            for elements in instrument_master:
                # print(elements.tag)
                new_dict.update({elements.tag:elements.text})
        ranking_details = doc.find(".//global_information/organization_information/classifications")
        if ranking_details is not None:
            for element in ranking_details:
                new_dict.update({element.tag + "_" + element.attrib["classification_ranking"]:element.text})
                
                
        
        
        global_information_rating =  doc.find(".//global_information/ratings")
        if global_information_rating is not None:
            for element in global_information_rating:
                tag = "_".join([element.attrib["agency"].replace("'",""),element.attrib["type"]])
                new_dict.update({tag.replace(" ","_"):element.text})
                new_dict.update({tag+"_"+"effective_date":element.attrib["effective_date"]})

        country_information = doc.find(".//country_information/instrument_country_information")
        if country_information is not None:
            for element in country_information:
                new_dict.update({element.tag:element.text})
        # print(bbid.attrib)instrument_details
        maturity_details = doc.find(".//instrument_details/maturity_details")

        if maturity_details is not  None:
            for element in maturity_details:
                new_dict.update({element.tag:element.text})
            
        increment_details = doc.find(".//instrument_details/denomination_amounts")
        if increment_details is not None:
            for element in increment_details:
                new_dict.update({element.tag:element.text})
         #muni_escrow       
        
        put_details = doc.find(".//debt/put_details")
        if put_details is not None:
            for element in put_details:
                new_dict.update({element.tag:element.text})
                
        
        
        escrow_details = doc.find(".//instrument_details/muni_escrow")
        if escrow_details is not None:
            for element in escrow_details:
                new_dict.update({element.tag:element.text})
        
        organization_master = doc.find(".//organization_master")
        if organization_master is not None:
            new_dict.update({"organization_master_id":organization_master.attrib["id"]})
            for element in organization_master:
#                 print(element.tag)
                if element.tag != "organization_xref":
                    new_dict.update({"issuer"+"_"+element.tag:element.text})
                else:
                    for childs in element:
                        new_dict.update({childs.attrib["entity_level"] + childs.attrib["entity_level"].replace(" ","_"):childs.text})
                        
            

        # for element in maturity_details:

        for elements in ["id_bb_sec_num","market_sector","security_typ","ticker","unique_id"]:
            new_dict.update({elements:bbid.attrib[elements]})
        debt_info = doc.find(".//debt/fixed_income")
        if debt_info is not None:
            for element in debt_info:
                new_dict.update({element.tag:element.text})
           
        muni_info = doc.find(".//debt/muni_details")
        
        if muni_info is not None:
            for element in muni_info:
                if element.tag != "linkage_child":
                    new_dict.update({element.tag:element.text})
        
        call_details_info = doc.find(".//debt/call_details")
        if call_details_info is not None:
            for element in call_details_info:
                if element.tag != "call_schedule":
                    new_dict.update({element.tag:element.text})
        floating_rate_details = doc.find(".//debt/floating_rate_instruments")
        if floating_rate_details is not None:
            for element in floating_rate_details:
                new_dict.update({element.tag:element.text})
                    
        tender_details = doc.find(".//debt/tender_details")
        if tender_details is not None:
            for element in tender_details:
                new_dict.update({element.tag:element.text})
            
        loc_details = doc.find(".//debt/loc_details")
        if loc_details is not None:
            for element in loc_details:
                new_dict.update({element.tag:element.text})
                
        sink_details = doc.find(".//debt/sink_details")
        if sink_details is not None:
            for element in sink_details:
                if element == "mandatory_sink_amount_type":
                    new_dict.update({element.tag:element.attrib["type"]})
                else:
                    new_dict.update({element.tag:element.text})

                # print(element.tag)
                # new_dict.update({element.tag:element.attrib["type"]})
#         <SIC classification_ranking="primary" classification_ranking_id="1" type="Standard Industrial Classification" type_id="1">6552</SIC>

    
        market_master = doc.find(".//market_master/market")
        if market_master is not None:
            new_dict.update({"primary_market":market_master.attrib["primary"]})
        
            for element in market_master:
                if element.tag == "xref":
                    new_dict.update({element.attrib["type"]:element.text})
                    new_dict.update({element.attrib["type"]+"_"+"status":element.attrib["status"]})
                else:
                    new_dict.update({element.tag:element.text})
                    

        amt_outstanding = doc.find(".//debt/amount_outstanding_history/change")
        # print(amt_outstanding.attrib)
        if amt_outstanding is not None:
            new_dict.update({"amount_outstanding_change_date":amt_outstanding.attrib["date"]})

            for elements in amt_outstanding:
                new_dict.update({elements.tag:elements.text})

        new_dict.update({cusip.attrib["type"]:cusip.text})
        new_dict.update({isin.attrib["type"]:isin.text})
        new_dict.update({bbid.attrib["type"]:bbid.text})


#         doc_dict.update(new_dict)
        
   
        yield new_dict

In [176]:
def get_call_schedule(author):
    for doc in author.iter('instrument'):
        # print(doc.attrib["id"])
        doc_dict = {}
        call_details_info = doc.find(".//debt/call_details")
        if call_details_info is not None:
            for element in call_details_info:
                if element.tag == "call_schedule":
#                 new_dict.update({element.tag:element.text})
                    new_dict = {}
                    new_dict.update({"instrument_id":doc.attrib["id"]})
                    new_dict.update({element.find(".//call_price").tag:element.find(".//call_price").text})
                    new_dict.update({element.find(".//call_date").tag:element.find(".//call_date").text})
                    if element.find(".//cav_call_price") is not None:
                        new_dict.update({element.find(".//cav_call_price").tag:element.find(".//cav_call_price").text})
                    yield new_dict
                       
                    
        
    

In [185]:
def get_default_schedule(author):
    for doc in author.iter('instrument'):
    # print(doc.attrib["id"])
        doc_dict = {}
        call_details_info = doc.find(".//global_information/instrument_details")
        if call_details_info is not None:
            for element in call_details_info:
                if element.tag == "default_status":
    #                 new_dict.update({element.tag:element.text})
                    new_dict = {}
                    new_dict.update({"instrument_id":doc.attrib["id"]})
                    new_dict.update({element.find(".//default_status").tag:element.find(".//default_status").text})
                    new_dict.update({element.find(".//default_status_date").tag:element.find(".//default_status_date").text})
                    yield new_dict

In [217]:
def partial_redemption_schedule(author):#partial_redemption
    for doc in author.iter('instrument'):
    # print(doc.attrib["id"])
        doc_dict = {}
        redemp_details_info = doc.iterfind(".//debt/partial_redemption")
        if redemp_details_info is not None:
            for element in redemp_details_info:
                new_dict = {}
                new_dict.update({"instrument_id":doc.attrib["id"]})
                if element.find(".//partial_redemption_amount") is not None:
                    new_dict.update({element.find(".//partial_redemption_amount").tag:element.find(".//partial_redemption_amount").text})
                if element.find(".//partial_redemption_date") is not None:
                    new_dict.update({element.find(".//partial_redemption_date").tag:element.find(".//partial_redemption_date").text})
                if element.find(".//partial_redemption_price") is not None:
                    new_dict.update({element.find(".//partial_redemption_price").tag:element.find(".//partial_redemption_price").text})
                if element.find(".//partial_redemption_type") is not None:
                    new_dict.update({element.find(".//partial_redemption_type").tag:element.find(".//partial_redemption_type").text})

                yield new_dict
    

In [197]:
def events(author):
    for doc in author.iter('instrument'):
    # print(doc.attrib["id"])

        call_details_info = doc.iterfind(".//debt/event")
        if call_details_info is not None:
            for element in call_details_info:
                new_dict = {}
                new_dict.update({"instrument_id":doc.attrib["id"]})
                new_dict.update({"code":element.attrib["code"]})
                new_dict.update({"date":element.attrib["date"]})
                new_dict.update({"type":element.attrib["type"]})
                new_dict.update({"event_text":element.text})
                yield new_dict


In [209]:
def get_sink_schedule(author):
    for doc in author.iter('instrument'):
        sink_schedule = doc.iterfind(".//debt/sink_schedule")
        if sink_schedule is not None:
            for element in sink_schedule:
                new_dict = {}
                new_dict.update({"instrument_id":doc.attrib["id"]})
                if element.find(".//mandatory_sink_amount") is not None:
                    new_dict.update({element.find(".//mandatory_sink_amount").tag:element.find(".//mandatory_sink_amount").text})
                if element.find(".//mandatory_sink_date") is not None:
                    new_dict.update({element.find(".//mandatory_sink_date").tag:element.find(".//mandatory_sink_date").text})
                if element.find(".//mandatory_sink_price") is not None:
                    new_dict.update({element.find(".//mandatory_sink_price").tag:element.find(".//mandatory_sink_price").text})

                yield new_dict

In [221]:
def get_organization_roles(author):
    for doc in author.iter('instrument'):
        inst_roles_schedule = doc.find(".//global_information/relations/instrument_roles")
        if inst_roles_schedule is not None:
            for element in inst_roles_schedule:
                new_dict = {}
                new_dict.update({"instrument_id":doc.attrib["id"]})
                new_dict.update({"id":element.attrib["id"]})
                new_dict.update({"organization":element.attrib["organization"]})
                new_dict.update({"type":element.attrib["type"]})
                new_dict.update({"text":element.text})
                yield new_dict

In [252]:
def amt_outstanding_history(author):
    for doc in author.iter('instrument'):
        amt_outstanding_schedule = doc.find(".//debt/amount_outstanding_history")
        if amt_outstanding_schedule is not None:
            for element in amt_outstanding_schedule:
                new_dict = {}
                new_dict.update({"instrument_id":doc.attrib["id"]})
                new_dict.update({"change_date":element.attrib["date"]})
                if element.find(".//amount_outstanding") is not None:
                    new_dict.update({element.find(".//amount_outstanding").tag:element.find(".//amount_outstanding").text})
                if element.find(".//amount_outstanding_decimal") is not None:
                    new_dict.update({element.find(".//amount_outstanding_decimal").tag:element.find(".//amount_outstanding_decimal").text})

                yield new_dict

In [259]:
def floating_rate_history(author):
    for doc in author.iter('instrument'):
        doc_dict = {}
        floating_rate = doc.iterfind(".//debt/floating_rate_history")
        if floating_rate is not None:
            for element in floating_rate:
                new_dict = {}
                new_dict.update({"instrument_id":doc.attrib["id"]})
                new_dict.update({"date":element.attrib["date"]})
                
                if element.find(".//rate") is not None:
                    new_dict.update({element.find(".//rate").tag:element.find(".//rate").text})
                if element.find(".//reset_frequency") is not None:
                    new_dict.update({element.find(".//reset_frequency").tag:element.find(".//reset_frequency").text})
                yield new_dict
    

In [240]:
tree.getroot()[0].find(".//timestamp").tag

'timestamp'

In [2]:
%timeit
tree = ET.parse(xml_data_string)
root = tree.getroot()[1]
time_stamp = tree.getroot()[0].find(".//timestamp").text
# doc_df = pd.DataFrame(list(iter_instrument(root,new_list_columns,count)))
doc_df = pd.DataFrame(list(iter_docs(root,time_stamp)))
# doc_df

NameError: name 'ET' is not defined

In [1]:
doc_df.head()


NameError: name 'doc_df' is not defined

In [222]:
organization_roles_df = pd.DataFrame(list(get_organization_roles(root)))

In [260]:
floating_rate_history_df = pd.DataFrame(list(floating_rate_history(root)))

In [267]:
doc_df.head()

,instrument_id,data_timestamp,apex_asset_type,instrument_type,primary_name,delivery_date,issue_price,settlement_date,issue_date,outstanding_indicator,...,loc_type,SIC_primary,NAICS_primary,call_cav,min_notification_days,reset_day,sink_defeased,other_enhancement_company,int_holiday_treatment_method,reset_holiday_treatment_method
0,347523459,2020-11-06T16:09:24Z,3,3,TAXABLE REF BDS 2020C,2020-11-18,100.0000000,2020-11-18,2020-11-18,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,347266315,2020-11-06T16:09:24Z,3,3,GO REF BDS 2020 D,2020-12-03,107.0800000,2020-12-03,2020-12-03,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,347532866,2020-11-06T16:09:24Z,3,3,GO REF BDS 2020 A,2020-11-30,113.0420000,2020-11-30,2020-11-30,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,347523472,2020-11-06T16:09:24Z,3,3,REF BDS 2020 A,2020-11-12,102.4440000,2020-11-12,2020-11-12,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185808825,2020-11-06T16:09:24Z,3,3,GO BDS 2018,2018-07-25,119.9013888,2018-07-25,2018-07-25,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
amt_outstanding_df = pd.DataFrame(list(amt_outstanding_history(root)))

In [257]:
amt_outstanding_df[amt_outstanding_df["instrument_id"] == "3587062"].head()
# print(len(amt_outstanding_df))
# amt_outstanding_df.head()

,instrument_id,change_date,amount_outstanding,amount_outstanding_decimal
844,3587062,2009-12-01,100000000,100000000
845,3587062,2016-05-01,96600000,96600000
846,3587062,2017-05-01,93000000,93000000
847,3587062,2018-05-01,89300000,89300000
848,3587062,2019-05-01,85400000,85400000


In [223]:
organization_roles_df.head()

,instrument_id,id,organization,type,text
0,347523459,2126956,"CREWS & ASSOCIATES, INC.",Lead Manager,3
1,347523459,2117326,"BOWLES, RICE, MCDAVID, GRAFF & LOVE (WV)",Bond Counsel,12
2,347266315,2124256,"D.A. DAVIDSON & ASSOC, INC.",Lead Manager,3
3,347266315,2146299,"UMB BANK AND TRUST, NA",Registrar,5
4,347266315,2146299,"UMB BANK AND TRUST, NA",Paying agent,4


In [219]:
partial_redemption_schedule_df.head()

,instrument_id,partial_redemption_amount,partial_redemption_date,partial_redemption_price,partial_redemption_type
0,5418103,1235000.0000000,2018-05-01,100.0000000,2
1,5418103,1315000.0000000,2019-05-01,100.0000000,2
2,5418103,1400000.0000000,2020-05-01,100.0000000,2
3,7386652,665000.0000000,2021-04-01,100.0000000,3
4,9820139,30000.0000000,2020-09-01,100.0000000,2


In [177]:
call_schedule_df = pd.DataFrame(list(get_call_schedule(root)))

In [198]:
events_df = pd.DataFrame(list(events(root)))

In [203]:
events_df.head()

,instrument_id,code,date,type,event_text
0,185808825,39,2018-07-18T12:00:42Z,3,BUDGET FYE 06/30/19
1,185808825,39,2018-07-18T12:00:44Z,3,BUDGET FYE 06/30/19
2,185808825,39,2018-07-18T12:00:47Z,3,BUDGET FYE 06/30/19
3,185808825,5,2019-01-02T13:01:01Z,3,CAFR FYE 06/30/18
4,185808825,5,2019-12-16T18:21:13Z,3,COMPREHENSIVE ANNUAL FINANC FYE 06/30/19


In [187]:
len(default_schedule_df)

5

In [179]:
call_schedule_df[call_schedule_df["instrument_id"] == "346215308"].head()
# call_schedule_df.head()

,instrument_id,call_price,call_date,cav_call_price
0,346215308,100.0000000,2028-11-01,NaN
1,346215308,100.0000000,2029-05-01,NaN
2,346215308,100.0000000,2029-11-01,NaN
3,346215308,100.0000000,2030-05-01,NaN
4,346215308,100.0000000,2030-11-01,NaN


In [243]:
doc_df.head()

,instrument_id,data_timestamp,apex_asset_type,instrument_type,primary_name,delivery_date,issue_price,settlement_date,issue_date,outstanding_indicator,...,loc_type,SIC_primary,NAICS_primary,call_cav,min_notification_days,reset_day,sink_defeased,other_enhancement_company,int_holiday_treatment_method,reset_holiday_treatment_method
0,347523459,2020-11-06T16:09:24Z,3,3,TAXABLE REF BDS 2020C,2020-11-18,100.0000000,2020-11-18,2020-11-18,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,347266315,2020-11-06T16:09:24Z,3,3,GO REF BDS 2020 D,2020-12-03,107.0800000,2020-12-03,2020-12-03,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,347532866,2020-11-06T16:09:24Z,3,3,GO REF BDS 2020 A,2020-11-30,113.0420000,2020-11-30,2020-11-30,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,347523472,2020-11-06T16:09:24Z,3,3,REF BDS 2020 A,2020-11-12,102.4440000,2020-11-12,2020-11-12,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185808825,2020-11-06T16:09:24Z,3,3,GO BDS 2018,2018-07-25,119.9013888,2018-07-25,2018-07-25,true,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
for columns in doc_df.columns:
    print(columns)

instrument_id
apex_asset_type
instrument_type
primary_name
delivery_date
issue_price
settlement_date
issue_date
outstanding_indicator
primary_currency_code
child_issue_ind
federal_tax_status
sector
sector_class
group
category
sub_category
Moodys_Long_Rating
Moodys_Long Rating_effective_date
Moodys_Credit_Watch_Long_Rating
Moodys_Credit Watch Long Rating_effective_date
Moodys_Issue_Long_Rating
Moodys_Issue Long Rating_effective_date
country_code
maturity_amount
denom_increment_amount
min_denom_amount
organization_master_id
issuerissuer
issuer_incorporated_state_code
issuer_primary_name
issuer_primary_name_abbreviated
issuer_organization_status
issuer_organization_type
issuer_org_country_code
id_bb_sec_num
market_sector
security_typ
ticker
unique_id
accrual_date
bond_form
bond_insurance
coupon_type
current_coupon_rate
daycount_basis_type
debt_type
default_indicator
depository_type
first_coupon_date
interest_payment_frequency
issue_amount
last_period_accrues_from_date
maturity_date
next_c

In [272]:
new_columns = []
for columns in doc_df.columns:
    columns = columns.replace("&","")
    new_columns.append(columns.replace(" ","_"))
doc_df.columns = new_columns

In [ ]:

def new_function(x):
     column_name = x.replace("&","_")
     column_name = column_name.replace("(","")
     column_name = column_name.replace(")","")
     column_name = column_name.replace("-","")
     column_name = column_name.replace("/"," ")

     column_name = "_".join(column_name.split(" "))
     
     column_name = column_name.replace("(","_")
     return column_name

In [105]:
test = root.find(".//call_schedule")

In [106]:
for element in test:
    print(element.tag)

call_date
call_price


In [20]:
test.text

'3'

In [88]:
doc_df["maturity_amount"]

0       2105000.0000000
1        260000.0000000
2        300000.0000000
3        910000.0000000
4      12855000.0000000
             ...       
787    75000000.0000000
788    46250000.0000000
789    60000000.0000000
790    42865000.0000000
791    10155000.0000000
Name: maturity_amount, Length: 792, dtype: object

In [397]:
doc_df["amount_outstanding"]

0       2105000
1        260000
2        300000
3        910000
4      12855000
         ...   
787    71365000
788    46250000
789    60000000
790    42865000
791    10155000
Name: amount_outstanding, Length: 792, dtype: object

In [91]:
bqclient = bigquery.Client(project=PROJECT,)
# bqstorageclient = bigquery_storage.BigQueryReadClient()

In [69]:
def iter_docs(author):
    author_attr = author.attrib
    for doc in author.iter('instrument'):
        # print(doc.attrib["id"])
        new_dict = {}
        new_dict.update({"id":doc.attrib["id"]})
        doc_dict = author_attr.copy()
        # xref = doc.find("master_information").find("instrument_xref")
        # print(xref.tag)

        cusip = doc.find('.//instrument_xref/xref[@type="CUSIP"]')
        isin = doc.find('.//instrument_xref/xref[@type="ISIN"]')
        bbid = doc.find('.//instrument_xref/xref[@type="Bloomberg Global Id"]')
        instrument_master = doc.find(".//instrument_master")
        

        for elements in instrument_master:
            # print(elements.tag)
            new_dict.update({elements.tag:elements.text})
        global_information_rating =  doc.find(".//ratings")
        if global_information_rating is None:
            pass
            # new_dict.update({"Moody's_Long Rating":None})
            # new_dict.update({"Moody's_Credit Watch Long Rating":None})
            # new_dict.update({"Moody's_Issue Long Rating":None})
            # new_dict.update({"effective_date":None})
        else:

            for element in global_information_rating:
                tag = "_".join([element.attrib["agency"].replace("'",""),element.attrib["type"]])
                new_dict.update({tag.replace(" ","_"):element.text})
                new_dict.update({"effective_date":element.attrib["effective_date"]})
        country_information = doc.find(".//country_information/instrument_country_information")
        for element in country_information:
            new_dict.update({element.tag:element.text})
        # print(bbid.attrib)instrument_details
        maturity_details = doc.find(".//instrument_details/maturity_details")
        market_master = doc.find(".//market_master")
        if maturity_details is None:
            pass
        else:
            for element in maturity_details:
                new_dict.update({element.tag:element.text})
        tender_details = doc.find(".//debt/tender_details")
        if tender_details is None:
            pass
        else:
            for element in tender_details:
                new_dict.update({element.tag:element.text})
        
        increment_details = doc.find(".//instrument_details/denomination_amounts")
        if increment_details is None:
            pass
        else:
            for element in increment_details:
                new_dict.update({element.tag:element.text})
        # for element in maturity_details:

        for elements in ["id_bb_sec_num","market_sector","security_typ","ticker","unique_id"]:
            new_dict.update({elements:bbid.attrib[elements]})
        debt_info = doc.find(".//debt/fixed_income")
        if debt_info is None:
            pass
        else:
            for element in debt_info:
                new_dict.update({element.tag:element.text})

        muni_info = doc.find(".//debt/muni_details")
        if muni_info is None:
            pass
        else:
            for element in muni_info:
                new_dict.update({element.tag:element.text})
        

        sink_details = doc.find(".//debt/sink_details")
        if sink_details is None:
            pass
        else:
            for element in sink_details:
                if element == "mandatory_sink_amount_type":
                    new_dict.update({element.tag:element.attrib["type"]})
                else:
                    new_dict.update({element.tag:element.text})

                # print(element.tag)
                # new_dict.update({element.tag:element.attrib["type"]})
        organization_master = doc.find(".//organization_master")
        if organization_master is not None:
            new_dict.update({"organization_master_id":organization_master.attrib["id"]})
            for element in organization_master:
                if element.tag != "organization_xref":
                    new_dict.update({"issuer"+"_"+elements.tag:elements.text})
                    

        amt_outstanding = doc.find(".//debt/amount_outstanding_history/change")
        # print(amt_outstanding.attrib)
        if amt_outstanding is None:
            pass
        else:
            new_dict.update({"amount_outstanding_change_date":amt_outstanding.attrib["date"]})

            for elements in amt_outstanding:
                new_dict.update({elements.tag:elements.text})

        new_dict.update({cusip.attrib["type"]:cusip.text})
        new_dict.update({isin.attrib["type"]:isin.text})
        new_dict.update({bbid.attrib["type"]:bbid.text})


        doc_dict.update(new_dict)
        
   
        yield doc_dict

In [92]:
def create_table_with_schema(bq,project_id,dataset,table_id,schema = []):
    PROJECT = project_id
    bq = bq
    table_id = '{}.{}.{}'.format(PROJECT,dataset,table_id)
    table = bq.create_table(table_id, exists_ok=True)
    print('{} created on {}'.format(table.table_id, table.created))
    table = bq.get_table(table_id)
    table.schema = schema
    table = bq.update_table(table, ["schema"])

def load_data(bq,data,project,dataset,table):
    bq = bq
    table_id = '{}.{}.{}'.format(project,dataset,table)
    job_config = bigquery.LoadJobConfig(schema =[])
    job = bq.load_table_from_dataframe(data, table_id,job_config=job_config)

    try:
        job.result() # Waits for the job to complete.
        return 'success'  
    except BadRequest as ex:
        print(ex) 

In [93]:
def load_data(bq,data,project,dataset,table):
    bq = bq
    table_id = '{}.{}.{}'.format(project,dataset,table)
    job_config = bigquery.LoadJobConfig(schema =[])
    job = bq.load_table_from_dataframe(data, table_id,job_config=job_config)

    try:
        job.result() # Waits for the job to complete.
        return 'success'  
    except BadRequest as ex:
        print(ex) 

In [268]:
PROJECT = 'eng-reactor-287421'
dataset = 'ICE'
table = 'ice_reference_data'

In [269]:
create_table_with_schema(bqclient,PROJECT,dataset,table,)
load_data(bqclient,doc_df,PROJECT,dataset,table)

ice_reference_data created on 2020-11-20 21:23:16.464000+00:00


In [273]:
load_data(bqclient,doc_df,PROJECT,dataset,table)

'success'

In [550]:
data_dict.head()


,Columns,Url
0,Instrument Identifier,/payload/instrument/@id
1,Change Date,/payload/instrument/debt/amount_outstanding_hi...
2,Amount Outstanding,/payload/instrument/debt/amount_outstanding_hi...
3,Call Identifier,/payload/instrument/debt/call_details/@num
4,Callable Compound Accreted Value (CAV) Indicator,/payload/instrument/debt/call_details/call_cav


In [551]:
data_dict["Url"].unique()[:5]

array(['/payload/instrument/@id',
       '/payload/instrument/debt/amount_outstanding_history/change/@date',
       '/payload/instrument/debt/amount_outstanding_history/change/amount_outstanding',
       '/payload/instrument/debt/call_details/@num',
       '/payload/instrument/debt/call_details/call_cav'], dtype=object)

In [450]:
test = root.find(".//payload/instrument")

In [486]:
test.attrib["id"]

'347523459'

In [642]:
path = ".//payload/instrument/debt/amount_outstanding_history/change/amount_outstanding"

In [643]:
node = root.find(path)
node.attrib

{}

In [520]:
test.split("/")

AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'split'

In [558]:
new_dict = {}
def url_function(url):
    # print(url)
    if "@" in url:
        url = url.split("/")
        dict_key = url[len(url)-1].replace("@","")
        final_dict_key =  url[len(url)-2] + "_" + dict_key
        # print(final_dict_key)
        url = url[1:len(url)-1]
        
        
        url.insert(0,"./",)
     
        url = "/".join(url)
        node = root.find(url)
        if node is not None:
            if len(node) !=0:
                new_dict.update({final_dict_key:node.attrib[dict_key]})
            else:
                new_dict.update({node.tag:node.text})
    else:
        url = "./"+url
        node = root.find(url)
        if node is not None:
            new_dict.update({node.tag:node.text})

        





In [559]:
data_dict["new_column_name"] = data_dict(lambda x : new_function(x["Columns"]),axis = 1)

TypeError: 'DataFrame' object is not callable

In [545]:
# data["new_column_name"].unique()

In [560]:
data["new_url"] = data.apply(lambda x : url_function(x["Url"]),axis = 1)

In [561]:
new_dict

{'instrument_id': '347523459',
 'change_date': '2020-11-18',
 'amount_outstanding': '2105000',
 'call_details_num': '347528327',
 'call_cav': 'true',
 'call_notice': '30',
 'call_timing': '1',
 'call_timing_in_part': '1',
 'extraordinary_make_whole_call': 'false',
 'extraordinary_redemption': 'true',
 'make_whole_benchmark': '9',
 'make_whole_call': 'false',
 'make_whole_call_end_date': '2038-05-01',
 'make_whole_call_spread': '12.5',
 'mandatory_redemption_code': '0',
 'maximum_call_notice_period': '60',
 'next_call_date': '2028-11-01',
 'next_call_price': '100.0000000',
 'optional_redemption_code': '0',
 'par_call_date': '2028-11-01',
 'par_call_price': '100.0000000',
 'accrual_date': '2020-11-18',
 'bond_form': '3',
 'bond_insurance': 'NONE',
 'coupon_type': '8',
 'current_coupon_rate': '1.7280000',
 'current_coupon_reset_date': '2020-11-06',
 'daycount_basis_type': '3',
 'debt_type': '3',
 'default_indicator': 'false',
 'depository_type': '1',
 'first_coupon_date': '2021-06-01',
 '